In [ ]:
let map = 
  File.ReadAllLines "input.txt"
  |> array2D
  |> Array2D.map (fun c -> int c - int '0')

let exampleMap = 
  @"1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581".Split('\n') |> array2D |> Array2D.map (fun c -> int c - int '0')


In [ ]:
let inBounds map (x,y) = x >= 0 && y >= 0 && x < Array2D.length1 map && y < Array2D.length2 map
let neighbors map (x,y) = [x + 1, y; x-1, y; x, y+1; x, y-1] |> Seq.filter (inBounds map)

let minRisk m start = 
  let h (x,y) = 198 - x - y
  let prioritize x lst = 
    match List.tryFindIndex (fun y -> snd y > snd x) lst with
    | Some idx -> List.insertAt idx x lst
    | None -> lst @ [x]
  
  let rec search (next :: boundary) =
    match next with 
    | ((x, y), cost) when x = Array2D.length1 m - 1 && y = Array2D.length2 m - 1 -> cost
    | (point, currentCost) -> 
      neighbors m point
      |> Seq.map (fun (x,y) -> ((x,y), currentCost + m[x,y]))
      |> Seq.fold (fun b (pt, cost) -> 
        match List.tryFindIndex (fun (p, _) -> p = pt) b with
        | Some idx when snd b[idx] > cost -> prioritize (pt, cost) (b |> List.removeAt idx)
        | Some _ -> b
        | None -> prioritize (pt, cost) b) boundary
      |> List.sortBy (fun (p,c) -> h p + c)
      |> search
  search [start, 0]

minRisk map (0,0)

In [ ]:
// A* Search pulled from NuGet
#r "nuget:astar-search"

let bigSearch m =
  let len = Array2D.length1 m
  let bigCost = function 0,0 -> 0 | x,y -> (m[x % len, y % len] + (x/len) + (y/len) - 1) % 9 + 1
  let between min maxExclusive n = n >= min && n < maxExclusive
  let bigNeighbors (x,y) = [x + 1, y; x-1, y; x, y+1; x, y-1] |> Seq.filter(fun (a,b) -> [a;b] |> Seq.forall (between 0 (5*len)))

  AStar.search (0,0) (5 * len - 1, 5 * len - 1) {
    fCost = fun _ (x,y) -> float (10 * len - 2 - x - y);
    gCost = fun _ pt -> bigCost pt |> float;
    neighbours = bigNeighbors;
    maxIterations = None
  } |> Option.map (Seq.sumBy bigCost)

bigSearch map

Installed Packages astar-search, 1.0.2

Value
2864
